# Generate data for DAVE2 training DONKEY
This notebook prepares data to train DAVE2

## Usage:
Run all cells

## Requirements:
./content/datasets/data_in/objects_testing/real/tub320x240_train/

## Outputs:
./content/datasets/h5_out_DRIVE_train/gt_real.h5 \
./content/datasets/h5_out_DRIVE_train/raw_image_real \
./content/datasets/pkl_out_DRIVE_train/gt_real.pkl \
./content/datasets/pkl_out_DRIVE_train/raw_image_real.pkl

In [ ]:
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import h5py
import numpy as np
import re
import json

In [14]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def map_values(value):
    if value in [128]:
        return 1
    elif value in [255]:
        return 2
    else:
        return 0
    return value

def map_to_r(number):
    colorr = [255, 128, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 255, 128, 255, 0, 128]
    if number<20:
      return int(colorr[number])
    else:
      return 0

def map_to_g(number):

    colorg = [0, 128, 255, 128, 0, 0, 128, 0, 0, 128, 0, 128, 0, 128, 255, 128, 0, 128, 255, 128]

    if number<20:
      return int(colorg[number])
    else:
      return 0

def map_to_b(number):

    colorb = [0, 0, 0, 128, 128, 255, 255, 128, 0, 128, 128, 255, 128, 0, 0, 0, 128, 255, 255, 128]

    if number<20:
      return int(colorb[number])
    else:
      return 255

def load_data(images_folder):
  raw_image=[]
  gt=[]
  image_files = sorted(os.listdir(images_folder),key=natural_sort_key)
  for image_file in image_files:
      if image_file.split(".")[-1]=="jpg":
          image = cv2.imread(images_folder+image_file)
          raw_image.append(image)
      if image_file.split(".")[-1]=="json" and image_file!='meta.json':
          with open(images_folder+image_file, 'r') as json_file:
            data = json.load(json_file)
            user_angle = float(data.get('user/angle', None))
            gt.append(user_angle)
  return raw_image,gt


raw_image_sim = []
raw_image_real = []


print("-STARTING real- ")
images_folder = f"./content/datasets/data_in/objects_testing/real/tub320x240_train/"
raw_image_real,gt_real=load_data(images_folder)

dataset_index="0001"
def find_lowest_dimensions(images_list1, images_list2):
    # Find the lowest height and width across both lists
    lowest_height = min(images_list1[0].shape[0], images_list2[0].shape[0])
    lowest_width = min(images_list1[0].shape[1], images_list2[0].shape[1])
    return lowest_height, lowest_width

def crop_images_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[100:, :lowest_width,:] for image in images_list]
    return cropped_images

lowest_height=140
lowest_width=10000
height, width = find_lowest_dimensions(raw_image_real, raw_image_real)
lowest_height=min(height,lowest_height)
lowest_width=min(width,lowest_width)
print(lowest_height,lowest_width)


raw_images_real={}
raw_images_real[dataset_index] = crop_images_to_lowest_dimensions(raw_image_real, lowest_height, lowest_width)
raw_gt_real={}
raw_gt_real[dataset_index]=gt_real



-STARTING real- 
140 320


In [ ]:
for i in range(0, 100,1):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 1, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs.imshow(cv2.cvtColor(raw_images_real[dataset_index][i], cv2.COLOR_BGR2RGB))
      axs.set_title(str(raw_gt_real[dataset_index][i]))
      
      axs.set_xticks([])
      axs.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [16]:
import pickle

with open('./content/datasets/pkl_out_DRIVE_train/raw_image_real.pkl', 'wb') as file:
    pickle.dump(raw_images_real, file)

with open('./content/datasets/pkl_out_DRIVE_train/gt_real.pkl', 'wb') as file:
    pickle.dump(raw_gt_real, file)

In [17]:
with open('./content/datasets/pkl_out_DRIVE_train/raw_image_real.pkl', 'rb') as file:
    loaded_images_real = pickle.load(file)
with open('./content/datasets/pkl_out_DRIVE_train/gt_real.pkl', 'rb') as file:
    loaded_gt_real = pickle.load(file)

In [ ]:
for i in range(0, 100,10):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 1, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs.imshow(cv2.cvtColor(loaded_images_real["0001"][i], cv2.COLOR_BGR2RGB))
      axs.set_title(str(loaded_gt_real["0001"][i]))
      axs.set_xticks([])
      axs.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [19]:
def save_dictionary_to_h5(file_path, data_dict):
    with h5py.File(file_path, 'w') as hf:
        for key, array_list in data_dict.items():
            # Create a group for each key in the dictionary
            group = hf.create_group(key)
            for i, arr in enumerate(array_list):
                # Save each array as a dataset within the group
                group.create_dataset(f'array_{i}', data=arr)



file_path = './content/datasets/h5_out_DRIVE_train/gt_real.h5'
save_dictionary_to_h5(file_path, loaded_gt_real)

file_path = './content/datasets/h5_out_DRIVE_train/raw_image_real.h5'
save_dictionary_to_h5(file_path, loaded_images_real)

In [20]:
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

def load_h5_to_dictionary(file_path):
    data_dict = {}
    with h5py.File(file_path, 'r') as hf:
        for key in hf.keys():
            # Create a list to store the arrays for each key
            array_list = []
            # Get the group corresponding to the current key
            group = hf[key]
            for dataset_name in group.keys():
                # Append each dataset (array) to the array list
                array_list.append(np.array(group[dataset_name]))
            # Store the array list for the current key in the dictionary
            data_dict[key] = array_list
    return data_dict

In [21]:
file_path = './content/datasets/h5_out_DRIVE_train/gt_real.h5'
loaded_dictionary_gt_real=load_h5_to_dictionary(file_path)

file_path = './content/datasets/h5_out_DRIVE_train/raw_image_real.h5'
loaded_dictionary_images_real=load_h5_to_dictionary(file_path)


In [22]:
def find_lowest_dimensions(images_list1, images_list2):
    # Find the lowest height and width across both lists
    lowest_height = min(images_list1[0].shape[0], images_list2[0].shape[0])
    lowest_width = min(images_list1[0].shape[1], images_list2[0].shape[1])
    return lowest_height, lowest_width

def crop_images_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[:lowest_height, :lowest_width,:] for image in images_list]
    return cropped_images


lowest_height=10000
lowest_width=10000


height, width = find_lowest_dimensions(loaded_dictionary_images_real["0001"], loaded_dictionary_images_real["0001"])
lowest_height=min(height,lowest_height)
lowest_width=min(width,lowest_width)
print(lowest_height,lowest_width)



loaded_dictionary_images_real["0001"] = crop_images_to_lowest_dimensions(loaded_dictionary_images_real["0001"], lowest_height, lowest_width)


140 320


In [ ]:
# Define the desired semantic ID you want to display
for i in range(1, 100):
  # Create a 2x2 grid of subplots
  fig, axs = plt.subplots(1, 1, figsize=(15, 6))
  plt.subplots_adjust(wspace=0.2, hspace=0.4)


  # Plot the real and fake images side by side in the first row
  axs.imshow(cv2.cvtColor(loaded_dictionary_images_real["0001"][i], cv2.COLOR_BGR2RGB))
  axs.set_title(str(loaded_dictionary_gt_real["0001"][i]))

  axs.set_xticks([])
  axs.set_yticks([])

  # Show the plot for the current iteration
  plt.show()